# Quantum State Tomography

* **Last Updated:** Jan 24, 2019
* **Requires:** qiskit-terra 0.7

In [64]:
# Needed for functions
import numpy as np
import time

# Import Qiskit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo
import measurement_correction as mc

## 2-Qubit state tomography Example

In [65]:
# Create a state preparation circuit
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.h(q2[0])
bell.cx(q2[0], q2[1])
print(bell.qasm())

job = qiskit.execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_statevector(bell)
print(psi_bell)

OPENQASM 2.0;
include "qelib1.inc";
qreg q8[2];
h q8[0];
cx q8[0],q8[1];

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [66]:
# Generate circuits and run on simulator
t = time.time()
qst_bell = tomo.state_tomography_circuits(bell, q2)
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
print('Time taken:', time.time() - t)

# Extract tomography data so that countns are indexed by measurement configuration
# Note that the None labels are because this is state tomography instead of process tomography
# Process tomography would have the preparation state labels there

tomo_counts_bell = tomo.tomography_data(job.result(), qst_bell)
tomo_counts_bell

[(QuantumRegister(2, 'q8'), 0), (QuantumRegister(2, 'q8'), 1)]
Time taken: 0.21445798873901367


{('X', 'X'): {'11': 2546, '00': 2454},
 ('X', 'Y'): {'11': 1288, '00': 1246, '10': 1269, '01': 1197},
 ('X', 'Z'): {'11': 1230, '00': 1256, '10': 1206, '01': 1308},
 ('Y', 'X'): {'11': 1222, '00': 1221, '10': 1241, '01': 1316},
 ('Y', 'Y'): {'10': 2516, '01': 2484},
 ('Y', 'Z'): {'11': 1226, '00': 1239, '10': 1250, '01': 1285},
 ('Z', 'X'): {'11': 1222, '00': 1262, '10': 1230, '01': 1286},
 ('Z', 'Y'): {'11': 1309, '00': 1242, '10': 1205, '01': 1244},
 ('Z', 'Z'): {'11': 2508, '00': 2492}}

In [67]:
# Generate fitter data and reconstruct density matrix
probs_bell, basis_matrix_bell, weights_bell = tomo.fitter_data(tomo_counts_bell)
rho_bell = tomo.state_cvx_fit(probs_bell, basis_matrix_bell, weights_bell)
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity =', F_bell)

Fit Fidelity = 1.001476063465005


### Repeat the Example with Measurement Noise

In [68]:
#Add measurement noise
noise_model = noise.NoiseModel()
for qi in range(2):
    read_err = noise.errors.readout_error.ReadoutError([[0.75, 0.25],[0.1,0.9]])
    noise_model.add_readout_error(read_err,[qi])
    
#generate the calibration circuits
meas_calibs, state_labels = mc.measurement_calibration_circuits(q2)

backend = Aer.get_backend('qasm_simulator')
qobj_cal = qiskit.compile(meas_calibs, backend=backend, shots=15000)
qobj_tomo = qiskit.compile(qst_bell, backend=backend, shots=15000)

job_cal = backend.run(qobj_cal, noise_model=noise_model)
job_tomo = backend.run(qobj_tomo, noise_model=noise_model)

cal_matrix = mc.generate_calibration_matrix(job_cal.result(),state_labels)
tomo_counts_bell = tomo.tomography_data(job_tomo.result(), qst_bell)

# Generate fitter data and reconstruct density matrix
probs_bell, basis_matrix_bell, weights_bell = tomo.fitter_data(tomo_counts_bell)

#no correction
rho_bell = tomo.state_cvx_fit(probs_bell, basis_matrix_bell, weights_bell)
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity (no correction) =', F_bell)

#correct data
probs_bell = mc.remove_measurement_errors(probs_bell,state_labels,cal_matrix, method=1)
rho_bell = tomo.state_cvx_fit(probs_bell, basis_matrix_bell, weights_bell)
F_bell = state_fidelity(psi_bell, rho_bell)
print('Fit Fidelity (w/ correction) =', F_bell)

Fit Fidelity (no correction) = 0.5755975315301844
Fit Fidelity (w/ correction) = 0.9933350732625035


## Generating and fitting random states

We now test the functions on state generated by a circuit consiting of a layer of random single qubit unitaries u3

In [69]:
def random_u3_tomo(nq, shots):
    
    def rand_angles():
        return tuple(2 * np.pi * np.random.random(3) - np.pi)
    q = QuantumRegister(nq)
    circ = QuantumCircuit(q)
    for j in range(nq):
        circ.u3(*rand_angles(), q[j])
    job = qiskit.execute(circ, Aer.get_backend('statevector_simulator'))
    psi_rand = job.result().get_statevector(circ)
    
    qst_circs = tomo.state_tomography_circuits(circ, q)
    job = qiskit.execute(qst_circs, Aer.get_backend('qasm_simulator'),
                         shots=shots)
    tomo_counts = tomo.tomography_data(job.result(), qst_circs)
    probs, basis_matrix, weights = tomo.fitter_data(tomo_counts, beta=0.5)
    
    rho_cvx = tomo.state_cvx_fit(probs, basis_matrix, weights)
    rho_mle = tomo.state_mle_fit(probs, basis_matrix, weights)
    
    print('F fit (CVX) =', state_fidelity(psi_rand, rho_cvx))
    print('F fit (MLE) =', state_fidelity(psi_rand, rho_mle))

In [70]:
for j in range(5):
    print('Random single-qubit unitaries: set {}'.format(j))
    random_u3_tomo(3, 5000)

Random single-qubit unitaries: set 0
[(QuantumRegister(3, 'q9'), 0), (QuantumRegister(3, 'q9'), 1), (QuantumRegister(3, 'q9'), 2)]
F fit (CVX) = 0.9953924545676174
F fit (MLE) = 0.99723309492854
Random single-qubit unitaries: set 1
[(QuantumRegister(3, 'q10'), 0), (QuantumRegister(3, 'q10'), 1), (QuantumRegister(3, 'q10'), 2)]
F fit (CVX) = 0.9984151989561932
F fit (MLE) = 0.995258133493655
Random single-qubit unitaries: set 2
[(QuantumRegister(3, 'q11'), 0), (QuantumRegister(3, 'q11'), 1), (QuantumRegister(3, 'q11'), 2)]
F fit (CVX) = 0.982076641158238
F fit (MLE) = 0.9944811521707045
Random single-qubit unitaries: set 3
[(QuantumRegister(3, 'q12'), 0), (QuantumRegister(3, 'q12'), 1), (QuantumRegister(3, 'q12'), 2)]
F fit (CVX) = 1.0163010070403429
F fit (MLE) = 0.9959599687264642
Random single-qubit unitaries: set 4
[(QuantumRegister(3, 'q13'), 0), (QuantumRegister(3, 'q13'), 1), (QuantumRegister(3, 'q13'), 2)]
F fit (CVX) = 0.9837746801866011
F fit (MLE) = 0.9950054527937733


## 5-Qubit Bell State

In [71]:
# Create a state preparation circuit
q5 = QuantumRegister(5)
bell5 = QuantumCircuit(q5)
bell5.h(q5[0])
for j in range(4):
    bell5.cx(q5[j], q5[j + 1])

# Get ideal output state
job = qiskit.execute(bell5, Aer.get_backend('statevector_simulator'))
psi_bell5 = job.result().get_statevector(bell5)

# Generate circuits and run on simulator
t = time.time()
qst_bell5 = tomo.state_tomography_circuits(bell5, q5)
job = qiskit.execute(qst_bell5, Aer.get_backend('qasm_simulator'), shots=5000)

# Extract tomography data so that countns are indexed by measurement configuration
tomo_counts_bell5 = tomo.tomography_data(job.result(), qst_bell5)
probs_bell5, basis_matrix_bell5, weights_bell5 = tomo.fitter_data(tomo_counts_bell5)
print('Time taken:', time.time() - t)

[(QuantumRegister(5, 'q14'), 0), (QuantumRegister(5, 'q14'), 1), (QuantumRegister(5, 'q14'), 2), (QuantumRegister(5, 'q14'), 3), (QuantumRegister(5, 'q14'), 4)]
Time taken: 44.07296371459961


In [72]:
t = time.time()
rho_mle_bell5 = tomo.state_mle_fit(probs_bell5, basis_matrix_bell5, weights_bell5)
print('MLE Reconstruction')
print('Time taken:', time.time() - t)
print('Fit Fidelity:', state_fidelity(psi_bell5, rho_mle_bell5))

MLE Reconstruction
Time taken: 5.6894690990448
Fit Fidelity: 0.9939676052139124


In [73]:
t = time.time()
rho_sdp_bell5 = tomo.state_cvx_fit(probs_bell5, basis_matrix_bell5, weights_bell5, solver='CVXOPT')
print('SDP Reconstruction')
print('Time taken:', time.time() - t)
print('Fidelity:', state_fidelity(psi_bell5, rho_sdp_bell5))

SDP Reconstruction
Time taken: 58.40936613082886
Fidelity: 0.9999284690902617
